In [1]:
import torch

if torch.cuda.is_available():
    allocated_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # 기가바이트 단위로 변환
    reserved_memory = torch.cuda.max_memory_reserved() / (1024 ** 3)
    print("CUDA is available on this machine.")

    print(f"Allocated GPU memory: {allocated_memory:.2f} GB")
    print(f"Reserved GPU memory: {reserved_memory:.2f} GB")
else:
    print("CUDA is not available on this machine.")

True
12.1
CUDA is available on this machine.
Allocated GPU memory: 0.00 GB
Reserved GPU memory: 0.00 GB


In [1]:
import os
import time
import math
import re
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from scipy.spatial import distance

from preprocessing.single_image_enhance_tflite import zeroDCE
from ultralytics import YOLO

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
current_directory = os.getcwd()

deep_available = False
is_timekeeping = True
visualable = False
brightiable = True
savenable = True


''' 반도체 이미지 폴더 '''
# path = os.path.join(current_directory, '...')
# path = os.path.join(current_directory, 'input_imgs/demo/')
# path = os.path.join(current_directory, 'test_demo/')
# path = os.path.join(current_directory, 'input_imgs/OK/')
# path = os.path.join(current_directory, 'test_demo/')
path = os.path.join(current_directory, '3OK/')


''' 결과 저장 폴더 '''
# save_path = os.path.join(current_directory, '...')
# save_path = os.path.join(current_directory, 'saved_imgs/demo')
# save_path = os.path.join(current_directory, 'saved_imgs')
# save_path = os.path.join(current_directory, 'saved_imgs/OK')
# save_path = os.path.join(current_directory, 'saved_imgs/.test')
save_path = os.path.join(current_directory, 'saved_imgs/3OK')

In [2]:
# YOLO 모델 
YOLOv8_small_seg = YOLO("yolov8s-seg.pt")

def xyxy2xy(xyxy):
    x = xyxy[:,0] + xyxy[:,2]    
    y = xyxy[:,1] + xyxy[:,3]
    return torch.cat((y.unsqueeze(1), x.unsqueeze(1)), dim=1) / 2

In [3]:
def filter2D(img):
    # 커널 마스크 생성
    kernel = np.ones((3,3), dtype=np.float64) / 9. 
    
    return cv2.filter2D(img, -1, kernel)

def canny(img):
    # 채널 단일화
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    '''
    img, threshold1, threshold2
    img : 입력 이미지, 보통 그레이스케일 이미지 사용
    threshold1 : 하위 임계값, 이 값보다 작을 경우 무시됨
    threshold2 : 상위 임계값, 이 값보다 클 경우 확실한 edge로 간주
    '''
    edge = cv2.Canny(gray, 28, 37)

    # 빈 BGR 이미지 생성
    bgr_img = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)
    
    # 엣지 픽셀의 위치에 흰색[255, 255, 255]으로 표시
    bgr_img[edge != 0] = [255, 255, 255]

    return edge, bgr_img

def merge(origin, modified):
    background = Image.fromarray(origin)
    foreground = Image.fromarray(modified)
    background.paste(foreground, (0,0), foreground)

    return np.array(background)

In [4]:
def natural_sort_key(string):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', string)]

class Image_Loader(Dataset):
    def __init__(self, dir_path, brighter, transformer=None):
        """
        파라미터 종류:
        이미지 폴더 경로, 시각화 여부, 흑백대비 여부, 딥러닝 여부
        """
        imgs_paths = np.array([dir_path + img_path for img_path in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, img_path))])
        self.imgs_paths = np.array(sorted(imgs_paths, key=natural_sort_key))
        self.imgs_paths = self.imgs_paths[0:]
        
        self.brighter = brighter
  
        self.transformer = transformer

    def __len__(self):
        return len(self.imgs_paths)
        
    def __getitem__(self, index):
        canny_img = None
        merge_img = None
        
        img_path = self.imgs_paths[index]
        img = cv2.imread(img_path)
        original_filename = os.path.basename(img_path)  # 원본 이미지 이름 추출


        if self.brighter:
            img = zeroDCE(img_path=img_path).numpy()

            img = filter2D(img)

            canny_img, canny_with_bw = canny(img)
            merge_img = merge(img, canny_img)

            img = merge_img
        
        if self.transformer:
            img = torch.from_numpy(img)
            """
            딥러닝 시,
            np.numpy -> torch.tensor로 변환 + 관련 추가 전처리 작성
            """
            img = img.to(device)

        return img, original_filename

In [5]:
"""
12시 방향 range
"""
def range_12(img, points, roi, AF, masks, normality):
    tmp = normality
    normality = 1
    
    """
    ㄷ자 mid 방향 처리
    """
    x_mid = np.int16(np.around(roi[1] + 220))
    y_mid = np.int16(np.around(roi[0] - 1264))
    w_mid = np.int16(np.around(roi[1] + 550))
    h_mid = np.int16(np.around(roi[0] - 750))

    x_AF = np.int16(np.around(roi[1] + 220))
    y_AF = np.int16(np.around(roi[0] - 1200))
    w_AF = np.int16(np.around(roi[1] + 310))
    h_AF = np.int16(np.around(roi[0] - 937))
    
    # 12시 방향 범위 내의 점들 선택
    patch_idx = np.where(
        (points[:, 0] > x_mid) &\
        (points[:, 0] < w_mid) &\
        (points[:, 1] > y_mid) &\
        (points[:, 1] < h_mid)
    )
    patch_mid = points[patch_idx[0]]

    patch_idx = np.where(
        (patch_mid[:, 0] <= x_AF) |\
        (patch_mid[:, 0] >= w_AF) |\
        (patch_mid[:, 1] <= y_AF) |\
        (patch_mid[:, 1] >= h_AF)
    )
    patch_mid = patch_mid[patch_idx[0]]

    # 색상 조건에 맞는 점들만 선택
    patch_mid = [circle for circle in patch_mid if all(color <= 94 for color in img[circle[1], circle[0]])]

    # AF 세그멘테이션 탐지 및 처리
    if ('-AF_2,' in AF) or ('-AF_2s,' in AF):
        normality = range_af_2(AF, normality, patch_mid, masks)
    else:
        normality = 0

    # 12시 방향 결과 시각화
    visualize_result(img, patch_mid, x_mid, y_mid, w_mid, h_mid, normality)
    visualize_result(img, patch_mid, x_AF, y_AF, w_AF, h_AF, normality) 
  
    return np.min([normality, tmp])

In [36]:
"""
9시 방향 range
"""
def range_9(img, points, roi, AF, masks, normality):
    tmp = normality
    normality = 1   
    
    """
    Right 부분 처리
    """
    x_right = np.int16(np.around(roi[1] - 820))
    y_right = np.int16(np.around(roi[0] - 405)) # y < 402
    w_right = np.int16(np.around(roi[1] - 280))
    h_right = np.int16(np.around(roi[0] - 174))

    x_AF = np.int16(np.around(roi[1] - 640))
    y_AF = np.int16(np.around(roi[0] - 377))
    w_AF = np.int16(np.around(roi[1] - 560))
    h_AF = np.int16(np.around(roi[0] - 267))    
    
    patch_idx = np.where(
        (points[:, 0] > x_right) &\
        (points[:, 0] < w_right) &\
        (points[:, 1] > y_right) &\
        (points[:, 1] < h_right)
    )
    patch_right = points[patch_idx[0]]

    patch_idx = np.where(
        (patch_right[:, 0] <= x_AF) |\
        (patch_right[:, 0] >= w_AF) |\
        (patch_right[:, 1] <= y_AF) |\
        (patch_right[:, 1] >= h_AF)
    )
    patch_right = patch_right[patch_idx[0]]    
       
    patch_right = [circle for circle in patch_right if all(color < 83 for color in img[circle[1], circle[0]])]
    if len(patch_right) != 2:
        normality = 0
    else:
        #if ('-AF_1,' in AF) or ('-AF_1s,' in AF):
        if ('-AF_1,' in AF) or ('-AF_1s,' in AF):
            normality = range_af_1(AF, normality, masks)
        else:
            normality = 0

    # 9시 방향 right 결과 시각화
    visualize_result(img, patch_right, x_right, y_right, w_right, h_right, normality)
    visualize_result(img, patch_right, x_AF, y_AF, w_AF, h_AF, normality)

 
    return np.min([normality, tmp])

In [37]:
"""
6시 방향 range
"""
def range_6(img, points, roi, normality):
    tmp = normality
    normality = 1   
    
    x = np.int16(np.around(roi[1] + 330))
    y = np.int16(np.around(roi[0] + 350))
    w = np.int16(np.around(roi[1] + 645)) # 602 < w
    h = np.int16(np.around(roi[0] + 606))
    
    x_2 = np.int16(np.around(roi[1] + 624))
    y_2 = np.int16(np.around(roi[0] + 350))
    w_2 = np.int16(np.around(roi[1] + 645))
    h_2 = np.int16(np.around(roi[0] + 425))

    x_3 = np.int16(np.around(roi[1] + 631))
    y_3 = np.int16(np.around(roi[0] + 470))
    w_3 = np.int16(np.around(roi[1] + 633))
    h_3 = np.int16(np.around(roi[0] + 472))
    
    x_AF = np.int16(np.around(roi[1] + 330))
    y_AF = np.int16(np.around(roi[0] + 350))
    w_AF = np.int16(np.around(roi[1] + 360))
    h_AF = np.int16(np.around(roi[0] + 400))      
    
    patch_idx = np.where(
        (points[:, 0] > x) &\
        (points[:, 0] < w) &\
        (points[:, 1] > y) &\
        (points[:, 1] < h)
    )
    patch3 = points[patch_idx[0]]

    patch_idx = np.where(
        (patch3[:, 0] <= x_2) |\
        (patch3[:, 0] >= w_2) |\
        (patch3[:, 1] <= y_2) |\
        (patch3[:, 1] >= h_2)
    )
    patch3 = patch3[patch_idx[0]]  
    
    patch_idx = np.where(
        (patch3[:, 0] <= x_3) |\
        (patch3[:, 0] >= w_3) |\
        (patch3[:, 1] <= y_3) |\
        (patch3[:, 1] >= h_3)
    )
    patch3 = patch3[patch_idx[0]]  
    
    patch_idx = np.where(
        (patch3[:, 0] <= x_AF) |\
        (patch3[:, 0] >= w_AF) |\
        (patch3[:, 1] <= y_AF) |\
        (patch3[:, 1] >= h_AF)
    )
    patch3 = patch3[patch_idx[0]]    
    
    patch3 = [circle for circle in patch3 if all(color < 100 for color in img[circle[1], circle[0]])]

    # 유클리드 거리가 120 미만인 점들만 남기기
    patch3 = np.array(patch3)
    if len(patch3) > 1:
        dists = distance.cdist(patch3, patch3, 'euclidean')
        close_points = np.any((dists < 91) & (dists != 0), axis=1)
        patch3 = patch3[close_points]

    if len(patch3) != 2:
        normality = 0
    
    visualize_result(img, patch3, x, y, w, h, normality)
    visualize_result(img, patch3, x_AF, y_AF, w_AF, h_AF, normality)
    visualize_result(img, patch3, x_2, y_2, w_2, h_2, normality)
    visualize_result(img, patch3, x_3, y_3, w_3, h_3, normality)    

    return np.min([normality, tmp])


In [38]:
"""
3시 방향 range
"""
def range_3(img, points, roi, normality):
    tmp = normality
    normality = 1   
    
    x = np.int16(np.around(roi[1] + 1100))
    y = np.int16(np.around(roi[0] - 450))
    w = np.int16(np.around(roi[1] + 1500))
    h = np.int16(np.around(roi[0] - 151)) #150 <=h
    
    x_AF = np.int16(np.around(roi[1] + 1207))
    y_AF = np.int16(np.around(roi[0] - 156))
    w_AF = np.int16(np.around(roi[1] + 1209))
    h_AF = np.int16(np.around(roi[0] - 151))  
    
    patch_idx = np.where(
        (points[:, 0] > x) &\
        (points[:, 0] < w) &\
        (points[:, 1] > y) &\
        (points[:, 1] < h)
    )
    patch4 = points[patch_idx[0]]
    
    patch_idx = np.where(
        (patch4[:, 0] <= x_AF) |\
        (patch4[:, 0] >= w_AF) |\
        (patch4[:, 1] <= y_AF) |\
        (patch4[:, 1] >= h_AF)
    )
    patch4 = patch4[patch_idx[0]] 
    
    patch4 = [circle for circle in patch4 if all(color < 92 for color in img[circle[1], circle[0]])] # 92

    # 유클리드 거리가 120 미만인 점들만 남기기
    patch4 = np.array(patch4)
    if len(patch4) > 1:
        dists = distance.cdist(patch4, patch4, 'euclidean')
        close_points = np.any((dists < 140) & (dists != 0), axis=1)
        patch4 = patch4[close_points]

    if len(patch4) != 2:
        normality = 0
        
    visualize_result(img, patch4, x, y, w, h, normality)
    visualize_result(img, patch4, x_AF, y_AF, w_AF, h_AF, normality)

    return np.min([normality, tmp])

In [39]:
"""
9시 방향 left range에 사용됨
"""
def range_af_1(AF, normality,patch, masks):
    Vertical_Length1 = []
    low_threshold = 52
    high_threshold = 116
    mask_array = masks.data.cpu().numpy()
    for i, mask in enumerate(mask_array):
        if len(mask.shape) > 2:
            mask = mask.squeeze()
        vertical_length1 = np.sum(mask, axis=0).max()
        Vertical_Length1.append(vertical_length1)
    is_seg = True
    for a in Vertical_Length1:
        if a < low_threshold:
            is_seg = False
            
        elif a > high_threshold:
            is_seg = False
    idx = 0 
    if ('-AF_1,' in AF):
        idx = AF.index('-AF_1,') - 1
    elif ('-AF_1s,' in AF):
        idx = AF.index('-AF_1s,') - 1

    if AF[idx] != '1':
        AF[idx] = '1'

    if (int(AF[idx]) * 2) != 2:
        normality = 0

    if not is_seg:
        normality = 0  
    
    return normality

"""
12시 방향 range에 사용됨
"""
def range_af_2(AF, normality, patch1, masks):
    Vertical_Length = []
    threshold = 92
    mask_array = masks.data.cpu().numpy()
    for i, mask in enumerate(mask_array):
        if len(mask.shape) > 2:
            mask = mask.squeeze()
        vertical_length = np.sum(mask, axis=0).max()
        Vertical_Length.append(vertical_length)

    is_seg = False
    for a in Vertical_Length:
        if a > threshold:
            is_seg = True
            idx = 0
            if ('-AF_2,' in AF):
                idx = AF.index('-AF_2,') - 1
            elif ('-AF_2s,' in AF):
                idx = AF.index('-AF_2s,') - 1

            if AF[idx] != '1':
                AF[idx] = '1'

            if (len(patch1) + int(AF[idx]) * 3) != 5:
                normality = 0
                break

    if not is_seg:
        normality = 0

    return normality

In [40]:
"""
이미지 시각화 -> 원탐지 탐지 x이거나 roi 이상한 것들만 시각화하는 코드로 변경 필요
"""
def visualize_result(img, patch, x, y, w, h, normality):
    for itr in patch:
        cv2.circle(img, (itr[0], itr[1]), 1, (0, 100, 100), 3)
        cv2.circle(img, (itr[0], itr[1]), itr[2], (255, 0, 255), 2)
        cv2.putText(img, f'r={itr[2]}', (itr[0] - itr[2], itr[1] + itr[2]), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
        # cv2.putText(img, f'x=({itr[0]}, {itr[1]})', (itr[0], itr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1) # OIS 중심 좌표

    if normality == 1:
        line_color = (0, 255, 0)  
        line_thickness = 2
    else:
        line_color = (0, 0, 255) 
        line_thickness = 2
    
    cv2.line(img, (x, y), (x, h), line_color, line_thickness)
    cv2.line(img, (x, y), (w, y), line_color, line_thickness)
    cv2.line(img, (w, y), (w, h), line_color, line_thickness)
    cv2.line(img, (x, h), (w, h), line_color, line_thickness)

''' 메모리 사용량 줄이기 '''
def range_all(img, points, roi, AF, masks, normality):
    normality = range_12(img, points, roi, AF, masks, normality)
    normality = range_9(img, points, roi, AF, masks, normality)
    normality = range_6(img, points, roi, normality)
    normality = range_3(img, points, roi, normality)
    
    del points
    return normality

In [41]:
distance_zero_count = 0

def zoom(img, masks, target_distance=1200, bg_color=(255, 255, 255)):
    global distance_zero_count

    def adjust_image(img, masks, target_distance, bg_color):
        global distance_zero_count
        
        def euclidean_distance(point1, point2):
            return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
        
        if not (masks and hasattr(masks, 'xy')):
            distance_zero_count += 1
            img_center = (img.shape[1] // 2, img.shape[0] // 2)
            return img, img_center, 0

        centers = [(mask[:, 0].mean(), mask[:, 1].mean()) for mask in masks.xy if len(mask) > 1]
        img_center = (img.shape[1] // 2, img.shape[0] // 2)

        if len(centers) < 2:
            distance_zero_count += 1
            return img, img_center, 0

        distances_from_center = sorted([(math.sqrt((img_center[0] - x) ** 2 + (img_center[1] - y) ** 2), (x, y)) for x, y in centers])

        filtered_centers = []
        for i in range(len(distances_from_center)):
            for j in range(i + 1, len(distances_from_center)):
                distance_between_centers = euclidean_distance(distances_from_center[i][1], distances_from_center[j][1])
                if distance_between_centers > 200:
                    filtered_centers.append(distances_from_center[i][1])
                    filtered_centers.append(distances_from_center[j][1])
                    break
            if len(filtered_centers) >= 2:
                break

        if len(filtered_centers) < 2:
            distance_zero_count += 1
            return img, img_center, 0

        closest_centers = filtered_centers[:2]
        distance = euclidean_distance(*closest_centers)
        higher_point, lower_point = max(closest_centers, key=lambda p: p[1]), min(closest_centers, key=lambda p: p[1])
        intersection_point = (int(lower_point[0]), int(higher_point[1]))
        # cv2.circle(img, intersection_point, 10, (0, 255, 0), 20)

        dx, dy = img_center[0] - intersection_point[0], img_center[1] - intersection_point[1]
        moved_img = cv2.warpAffine(img, np.float32([[1, 0, dx], [0, 1, dy]]), (img.shape[1], img.shape[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=bg_color)

        if distance == 0:
            return moved_img, img_center, 0
        
        scale_factor = target_distance / distance
        new_size = (int(moved_img.shape[1] * scale_factor), int(moved_img.shape[0] * scale_factor))
        resized_img = cv2.resize(moved_img, new_size, interpolation=cv2.INTER_LINEAR)

        if scale_factor > 1:  # 확대
            x_offset, y_offset = (resized_img.shape[1] - moved_img.shape[1]) // 2, (resized_img.shape[0] - moved_img.shape[0]) // 2
            cropped_img = resized_img[y_offset:y_offset + moved_img.shape[0], x_offset:x_offset + moved_img.shape[1]]
            return cropped_img, intersection_point, distance
        else:  # 축소
            canvas = np.full_like(moved_img, bg_color, dtype=np.uint8)
            x_offset, y_offset = (moved_img.shape[1] - resized_img.shape[1]) // 2, (moved_img.shape[0] - resized_img.shape[0]) // 2
            canvas[y_offset:y_offset + resized_img.shape[0], x_offset:x_offset + resized_img.shape[1]] = resized_img
            return canvas, intersection_point, distance

    adjusted_img, _, _ = adjust_image(img, masks, target_distance, bg_color)
    return adjusted_img

def get_distance_zero_count():
    global distance_zero_count
    return distance_zero_count

In [42]:
def classification(img, save_info=False, visualizer=False):
    torch.cuda.empty_cache()
    [classes, img, masks] = YOLOv8_small_seg.predict(source=img, save=True, show_boxes=False, show_labels=False, show_conf=False, imgsz=1312, conf=0.6, retina_masks=False)
    torch.cuda.empty_cache()
    
    normality = 1
    roi = None
    if masks is None:
        normality = 0
    else:
        try:
            centers = [(mask[:, 0].mean(), mask[:, 1].mean()) for mask in masks.xy if len(mask) > 0 and len(mask[0]) > 1]
            img_center = (img.shape[1] // 2, img.shape[0] // 2)

            distances_from_center = sorted([(math.sqrt((img_center[0] - x) ** 2 + (img_center[1] - y) ** 2), (x, y)) for x, y in centers])

            filtered_centers = []
            for i in range(len(distances_from_center)):
                for j in range(i + 1, len(distances_from_center)):
                    distance_between_centers = math.sqrt((distances_from_center[i][1][0] - distances_from_center[j][1][0]) ** 2 + (distances_from_center[i][1][1] - distances_from_center[j][1][1]) ** 2)
                    if distance_between_centers > 200:
                        filtered_centers.append(distances_from_center[i][1])
                        filtered_centers.append(distances_from_center[j][1])
                        break
                if len(filtered_centers) >= 2:
                    break

            closest_centers = filtered_centers[:2]

            if len(closest_centers) > 1:
                higher_point = max(closest_centers, key=lambda p: p[1])
                lower_point = min(closest_centers, key=lambda p: p[1])
                intersection_point = (lower_point[0], higher_point[1])

                roi = (int(intersection_point[0]), int(intersection_point[1]))
                # cv2.circle(img, roi, 10, (0, 0, 255), 20)

        except IndexError:
            roi = None

    info = detection(img, method=Hough_Transform, brighted=brightiable, DL=deep_available)
    if info is not None and info.size > 0:
        info = np.uint16(np.around(info))
    else:
        pass

    if roi == None:
        normality = 0
    else:
        points = np.array(info[0][:, :3])
        normality = range_all(img, points, roi, classes, masks, normality)

    if visualizer:
        plt.figure(figsize=(16, 12))
        plt.imshow(img)
        plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
        plt.show()

        if normality == 0:
            print('불량')
        else:
            print('정상')

    if save_info:
        if (int(normality) + int(not 'NG' in path)) != 1:
            if not os.path.exists(save_path + '/True/'):
                os.makedirs(save_path + '/True/')
            cv2.imwrite(os.path.join(save_path + '/True/', original_filename), img)

        else:
            if not os.path.exists(save_path + '/Fake/'):
                os.makedirs(save_path + '/Fake/')
            cv2.imwrite(os.path.join(save_path + '/Fake/', original_filename), img)
    
    del img
    del info
    del roi
    del classes
    del masks

    return normality


In [43]:
def Hough_Transform(img, brighted):
    gray1 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.GaussianBlur(gray1, (9, 9), 2)
    
    """
    image, method, dp
    4) minDist : 검출된 원들 간의 최소 거리, 검출할 원의 중심 사이의 최소 거리로, 값이 작을수록 원이 더 많이 검출됨
    5) param1 : Canny edge 검출기의 상위 임계값
    6) param2 : 색상 민감도, 숫자가 작을수록 민감도가 떨어짐
    7) minRadius : 검출할 원의 최소 반지름
    8) maxRadius : 검출할 원의 최대 반지름
    """
    if brighted:
        return cv2.HoughCircles(gray2, cv2.HOUGH_GRADIENT, 1, minDist=25, param1=13, param2=19, minRadius=35, maxRadius=56) #22
    else:
        return cv2.HoughCircles(gray1, cv2.HOUGH_GRADIENT, 1, 20, param1=62, param2=34, minRadius=10, maxRadius=100)

def detection(img, method, brighted, DL):
    
    if DL:
        """
        딥러닝 알고리즘
        """
    else:
        return method(img, brighted)

In [44]:
img_loader = None

if deep_available:
    transform = transforms.ToTensor()
    img_loader = Image_Loader(dir_path=path, brighter=brightiable, transformer=transform)
else:
    img_loader = Image_Loader(dir_path=path, brighter=brightiable)
     
timekeeping = 0.0
keeped_times = 0.0
            
if is_timekeeping: 
    timekeeping = time.time() 

count = [0]
itr = 1
errorRate =0

for img, original_filename in img_loader:
    [classes, _, masks] = YOLOv8_small_seg.predict(source=img, save=True, show_boxes=True, show_labels=False, show_conf=False, imgsz=1312, conf=0.6, retina_masks=False)

    img = zoom(img, masks=masks)

    errorRate += classification(img, save_info=savenable, visualizer=visualable)
    
    if is_timekeeping: 
        keeped = time.time() - timekeeping
        if visualable:
            print(f'처리속도 : {keeped}초\n')
        elif itr <= 25:
            keeped_times += keeped
    
    if visualable:
        if not is_timekeeping: 
            print()
    else:
        if itr % 10 == 0:
            print(f'진행률 : {itr}/{len(img_loader)}')

    itr += 1
    
    if is_timekeeping: 
        timekeeping = time.time()

진행률 : 10/624
진행률 : 20/624
진행률 : 30/624
진행률 : 40/624
진행률 : 50/624
진행률 : 60/624
진행률 : 70/624
진행률 : 80/624
진행률 : 90/624
진행률 : 100/624
진행률 : 110/624
진행률 : 120/624
진행률 : 130/624
진행률 : 140/624
진행률 : 150/624
진행률 : 160/624
진행률 : 170/624
진행률 : 180/624
진행률 : 190/624
진행률 : 200/624
진행률 : 210/624
진행률 : 220/624
진행률 : 230/624
진행률 : 240/624
진행률 : 250/624
진행률 : 260/624
진행률 : 270/624
진행률 : 280/624
진행률 : 290/624
진행률 : 300/624
진행률 : 310/624
진행률 : 320/624
진행률 : 330/624


KeyboardInterrupt: 

In [ ]:
''' 결과 로그 출력 '''

if 'NG' in path:
    print(f'불량 탐지율 : {len(img_loader) - errorRate}/{len(img_loader)}')
else:
    print(f'정상 탐지율 : {errorRate}/{len(img_loader)}')

print(f'저장된 경로 : {save_path}')
print(f'평균 처리속도 : {keeped_times / 25.0}초')
print(f'AF 2개 미만 탐지 : {distance_zero_count}회')

불량 탐지율 : 251/252
저장된 경로 : c:\Users\user\Detection8\saved_imgs/3NG
평균 처리속도 : 0.4718578815460205초
AF 2개 미만 탐지 : 6회


In [ ]:
251/252

0.996031746031746